In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from functools import reduce
import operator

import pymongo

In [2]:
load_dotenv()

True

#### Paris: Fontain Agam. Melbourne: Australia Post. Sao Paulo: Sala Sao Paulo

In [22]:
paris_ld = {'type': 'Point', 'coordinates': ['48.8905568192578','2.242512906913018']}

In [23]:
melbourne_cbd = {'type': 'Point', 'coordinates': ['-37.81227175879301','144.96262186884945']}

In [24]:
sao_paulo_zc = {'type': 'Point', 'coordinates': ['-23.533215960575276','-46.639408758375076']}

In [25]:
def getdata_fsq(url, city, venue):
    client_id = os.getenv("tok1")
    client_secret = os.getenv("tok2")
    url_ = f'{url}'
    c = city['coordinates'][0]
    c1 = city['coordinates'][1]
    parametros = {"client_id": client_id, "client_secret": client_secret, "v": "20180323", "ll": f'{c}, {c1}',
                  "query": venue, "limit": 55}
    resp = requests.get(url_, params=parametros).json()
    return resp

#### HACER UNA FUNCION PARA GENERAR DATAS POR CADA RUBRO, POR CADA CIUDAD

In [ ]:
def func(url, )

In [26]:
datap = getdata_fsq('https://api.foursquare.com/v2/venues/search', paris_ld, 'vegan')

In [5]:
datam = getdata_fsq('https://api.foursquare.com/v2/venues/search', melbourne_cbd, 'airport')

In [6]:
datam_= datam['response']['venues']

In [27]:
datap_= datap['response']['venues']

In [30]:
def getFromDict(dic,map_):
    return reduce(operator.getitem, map_, dic)

In [31]:
def type_point(gps):
    return {"type": "Point", "coordinates": gps}

#### HACER FUNCION PARA GENERAR UN JSON POR CADA DATA agregar Tipo (vegan, starb, etc)

In [32]:
map_name = ['name']
map_city = ['location', 'city']
map_lat = ['location', 'lat']
map_long = ['location', 'lng']
gastr = []
for dic in datap_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["latitude"],ndic["longitude"]])
    gastr.append(ndic)

In [11]:
airp = []
for dic in datam_:
    ndic = {}
    ndic['name']= getFromDict(dic,map_name)
#    ndic['city']= getFromDict(dic,map_city)
    ndic['latitude']= getFromDict(dic,map_lat)
    ndic['longitude']= getFromDict(dic,map_long)
    ndic['location']= type_point([ndic["longitude"], ndic["latitude"]])
    airp.append(ndic)

#### HACER FUNCION PARA GENERAR LOS JSON

In [34]:
#def jsongenerator(name):
with open('vegans.json', 'w') as f:
    json.dump(gastr, f)

In [12]:
with open('airportsm.json', 'w') as f:
    json.dump(airp, f)

In [35]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")

In [36]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [17]:
from pymongo import GEOSPHERE

In [37]:
g = db.get_collection("gastronomy")

In [38]:
g.find_one().keys()

dict_keys(['_id', 'name', 'city', 'latitude', 'longitude', 'location'])

In [39]:
def type_point(lista):
    return {"type": "Point", "coordinates": lista}

In [40]:
coordenadas2 = [48.8905568192578, 2.242512906913018]

In [197]:
#paris_ld = {'type': 'Point', 'coordinates': ['48.8905568192578','2.242512906913018']}

In [48]:
query = {"location": {"$near": {"$geometry": type_point(coordenadas2), "$minDistance": 0 ,"$maxDistance": 8000}}}

In [49]:
list(g.find(query))

[{'_id': ObjectId('61323dcf6bc18e4f204f84d9'),
  'name': 'La Maison Végane',
  'city': 'Puteaux',
  'latitude': 48.879252,
  'longitude': 2.241002,
  'location': {'type': 'Point', 'coordinates': [48.879252, 2.241002]}},
 {'_id': ObjectId('61323dcf6bc18e4f204f84d1'),
  'name': 'Chacha Vegan Street Food',
  'city': 'Paris',
  'latitude': 48.881644,
  'longitude': 2.298055,
  'location': {'type': 'Point', 'coordinates': [48.881644, 2.298055]}}]